# 1 Basic inspection

In [5]:
# import all libraries
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [8]:
# Load the data
azdias = pd.read_csv('data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('data/Udacity_CUSTOMERS_052018.csv', sep=';')

Let's start by veryfing the shape of our datasets

In [9]:
print("Azdias shape -",azdias.shape)
print("Customers shape -",customers.shape)
extra_cols_in_customers = [col for col in customers.columns if col not in azdias.columns]
print("Extra columns in Customers Data - ",extra_cols_in_customers)

Azdias shape - (601875, 366)
Customers shape - (191652, 369)
Extra columns in Customers Data -  ['PRODUCT_GROUP', 'CUSTOMER_GROUP', 'ONLINE_PURCHASE']


Unfortunatly, even after several trials, the notebook instance never managed to upload the integrality of the Azdias dataset. Approximatly 200 000 thousands rows from the original dataset were lost.  
**Let's now focus on data treatment.**

Later, we will want to perform the same PCA transformer and clustering model on both datasets, let's normalize them now by getting rid of the customers extra columns.

In [10]:
del customers['PRODUCT_GROUP']
del customers['CUSTOMER_GROUP']
del customers['ONLINE_PURCHASE']